In [4]:
import os
import sys
import time
from datetime import datetime

import numpy as np
import pandas as pd

# Import from the parent directory
sys.path.append(os.path.join(os.getcwd(), ".."))
from teamwork import teamwork as tw
from utils.utils import *

In [11]:
"""Static Column Names"""
VISIT_ID = "visit_id"
ADMISSION_DATE = "admission_date"
NOTE_DATE = "note_date"
NOTE_AUTHOR = "note_author"
TEAM_CONDITION = "team_condition"
NORM_ADMISSION_DATE = "norm_admission_date"
NORM_NOTE_DATE = "norm_note_date"
IS_IN_TEAM = "is_in_team"
IS_IN_TEAM_X = f"{IS_IN_TEAM}_x"
IS_IN_TEAM_Y = f"{IS_IN_TEAM}_y"
EDGE = "edge"
IS_AFTER_DELTA = "is_after_delta"

default_columns = {
    VISIT_ID: "id",
    ADMISSION_DATE: "arrive_date",
    NOTE_DATE: "date",
    NOTE_AUTHOR: "dr",
    #new column for heart failure indicator
    TEAM_CONDITION: "hf"
}

_, admit_date, *_, team_condition = [*default_columns.values()]
print(team_condition)

hf


In [5]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%d-%m %H:%M:%S")

notes_table = pd.read_csv('../data/notes_6May2021_anonymous.csv', parse_dates=['note_date','admission_date'])
notes_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769429 entries, 0 to 769428
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   author          769429 non-null  int64 
 1   note_date       769284 non-null  object
 2   admission_date  769429 non-null  object
 3   id              769429 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 23.5+ MB


In [6]:
data = {
    0: {
        "age": 75,
        "arrive_date": "2019-01-01 00:00:00",
        "date": "2019-01-01 19:15:00",
        "discharge_date": "2019-01-01",
        "disposition": 1,
        "dr": "Brad Palmer",
        "id": 0,
        "patient": "patient1",
    },
    1: {
        "age": 68,
        "arrive_date": "2019-01-24 00:00:00",
        "date": "2019-01-24 10:19:00",
        "discharge_date": "2019-01-24",
        "disposition": 0,
        "dr": "Albert Romero",
        "id": 1,
        "patient": "patient2",
    },
    2: {
        "age": 68,
        "arrive_date": "2019-01-24 00:00:00",
        "date": "2019-01-24 17:09:00",
        "discharge_date": "2019-01-24",
        "disposition": 0,
        "dr": "Margie Meyer",
        "id": 1,
        "patient": "patient2",
    },
    3: {
        "age": 68,
        "arrive_date": "2019-01-24 00:00:00",
        "date": "2019-01-24 16:48:00",
        "discharge_date": "2019-01-24",
        "disposition": 0,
        "dr": "Evan Frazier",
        "id": 1,
        "patient": "patient2",
    },
    8: {
        "age": 71,
        "arrive_date": "2019-02-14 00:00:00",
        "date": "2019-02-14 20:58:00",
        "discharge_date": "2019-02-14",
        "disposition": 0,
        "dr": "Albert Romero",
        "id": 2,
        "patient": "patient3",
    },
    9: {
        "age": 71,
        "arrive_date": "2019-02-14 00:00:00",
        "date": "2019-02-14 15:54:00",
        "discharge_date": "2019-02-14",
        "disposition": 0,
        "dr": "Margie Meyer",
        "id": 2,
        "patient": "patient3",
    },
    12: {
        "age": 66,
        "arrive_date": "2019-04-15 19:15:00",
        "date": "2019-04-16 01:29:00",
        "discharge_date": "2019-04-15",
        "disposition": 0,
        "dr": "Albert Romero",
        "id": 6,
        "patient": "patient4",
    },
    13: {
        "age": 66,
        "arrive_date": "2019-04-15 19:15:00",
        "date": "2019-04-15 20:19:00",
        "discharge_date": "2019-04-15",
        "disposition": 0,
        "dr": "Margie Meyer",
        "id": 6,
        "patient": "patient4",
    },
    14: {
        "age": 66,
        "arrive_date": "2019-04-15 19:15:00",
        "date": "2019-04-16 04:43:00",
        "discharge_date": "2019-04-15",
        "disposition": 0,
        "dr": "Evan Frazier",
        "id": 6,
        "patient": "patient4",
    },
    15: {
        "age": 66,
        "arrive_date": "2019-04-15 19:15:00",
        "date": "2019-04-18 21:23:00",
        "discharge_date": "2019-04-15",
        "disposition": 0,
        "dr": "Myrtle George",
        "id": 6,
        "patient": "patient4",
    },
    16: {
        "age": 66,
        "arrive_date": "2019-04-15 19:15:00",
        "date": "2019-04-16 07:00:00",
        "discharge_date": "2019-04-15",
        "disposition": 0,
        "dr": "Victoria Washington",
        "id": 6,
        "patient": "patient4",
    },
    22: {
        "age": 69,
        "arrive_date": "2019-08-15 00:00:00",
        "date": "2019-08-15 17:07:00",
        "discharge_date": "2019-08-15",
        "disposition": 1,
        "dr": "Neil Mitchell",
        "id": 7,
        "patient": "patient5",
    },
}


In [7]:
test_df = pd.DataFrame.from_dict(data, orient="index").astype(
            {"date": "datetime64", "arrive_date": "datetime64"}
        )

In [8]:
test_df

,age,arrive_date,date,discharge_date,disposition,dr,id,patient
0,75,2019-01-01 00:00:00,2019-01-01 19:15:00,2019-01-01,1,Brad Palmer,0,patient1
1,68,2019-01-24 00:00:00,2019-01-24 10:19:00,2019-01-24,0,Albert Romero,1,patient2
2,68,2019-01-24 00:00:00,2019-01-24 17:09:00,2019-01-24,0,Margie Meyer,1,patient2
3,68,2019-01-24 00:00:00,2019-01-24 16:48:00,2019-01-24,0,Evan Frazier,1,patient2
8,71,2019-02-14 00:00:00,2019-02-14 20:58:00,2019-02-14,0,Albert Romero,2,patient3
9,71,2019-02-14 00:00:00,2019-02-14 15:54:00,2019-02-14,0,Margie Meyer,2,patient3
12,66,2019-04-15 19:15:00,2019-04-16 01:29:00,2019-04-15,0,Albert Romero,6,patient4
13,66,2019-04-15 19:15:00,2019-04-15 20:19:00,2019-04-15,0,Margie Meyer,6,patient4
14,66,2019-04-15 19:15:00,2019-04-16 04:43:00,2019-04-15,0,Evan Frazier,6,patient4
15,66,2019-04-15 19:15:00,2019-04-18 21:23:00,2019-04-15,0,Myrtle George,6,patient4


In [9]:
test_visit_id = 6
corpus = tw.TeamworkCorpus(test_df)
actual_edgesize = len(corpus.visit_id_to_edges_dict[test_visit_id])

KeyError: "['hf'] not in index"

In [12]:
G = corpus.team_experience_dict[6]['graph']

In [1]:
import matplotlib.pyplot as plt
import networkx as nx

# elarge = [(u, v, d) for (u, v, d) in G.edges(data=True) if d["weight"] > 0.5]
# esmall = [(u, v, d) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.5]

pos = nx.spring_layout(G, seed=5)  # positions for all nodes - seed for reproducibility
# nx.random_layout
# nodes
# nx.draw_networkx_nodes(G, pos, node_size=500)
nx.draw(G, pos, )
# edges
nx.draw_networkx_edges(G, pos, edgelist=elarge, width=4)
nx.draw_networkx_edges(
    G, pos, edgelist=esmall, width=4, alpha=0.5, edge_color="b", style="dashed"
)

# labels
nx.draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif")

ax = plt.gca()
ax.margins(0.1)
plt.axis("off")
plt.tight_layout()
plt.show()

NameError: name 'G' is not defined